### Find Word Images And Copy

In [25]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [26]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

file_ext = 1000

# image result parameter
select_image_num = 1  # 10 image sample number big and equal this number
take_image_sample_num = 1000  # 50

In [27]:
word_lemma_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-2-Word In Visual Genome Merge"

word_lemma_data_path2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-3-Find Word Images Mask Fade"

path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-4-Find Word Images And Copy"

#Path(path).mkdir(parents=True, exist_ok=True)

In [28]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [29]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [30]:
def take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50):
    '''take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then sort_target_column is sorting according to sort_ascending condition.\n
    Finally, taking sample_num each word_list values.\n 
    ex.\n
    take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat, word_list, "word", "search_text", sort_ascending=True, sample_num=50)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        df_select = df_source[df_source[f"{word_source_column}"] == word]
        df_select = df_select.sort_values(f"{sort_target_column}",key=lambda x:x.str.len(), ascending=sort_ascending).head(sample_num)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [31]:
#def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder):
#    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder)\n
#    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
#    output_path_folder from image_folder_path.\n
#    ex.\n
#    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
#    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
#    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", image_path, output_path_folder)
#    '''
#    df_search_result = pd.DataFrame()
#    for word in word_list:
#        path = f"{output_path_folder}/{word}"
#        Path(path).mkdir(parents=True, exist_ok=True)        
#        df_select = df_source[df_source[f"{word_source_column}"] == word]
#        for image_id in df_select[f"{image_id_column}"]:
#            image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
#            for l in image_file:
#                source = l # source directory
#                destination = path
#                shutil.copy2(source, destination)

In [32]:
def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, opt_column, image_folder_path, mask_folder_path, output_path_folder):
    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, mask_folder_path, output_path_folder)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
    output_path_folder from mask_folder_path (search in firstly) and image_folder_path. opt_column used for additional features\n
    ex.\n
    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path output_path_folder)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        try:
            path = f"{output_path_folder}/{word}"
            Path(path).mkdir(parents=True, exist_ok=True)        
            df_var = df_source[df_source[f"{word_source_column}"] == word]
            df_select = df_var.drop_duplicates(subset=[f"{image_id_column}"])  # ????
            imageid_list = list(df_select[f"{image_id_column}"])
            opt_column_list = list(df_select[f"{opt_column}"])
            data_list_zip = zip(imageid_list, opt_column_list)
            for image_id, opt in data_list_zip:
                opt_num = int(opt)

                #if isfile(f"{image_folder_path}/VG_Mask_Blur/{image_id}_{opt_num}_mask_blur.jpg"):  # ?????                
                #    image_file = glob.glob(f"{image_folder_path}/*/{image_id}_{opt_num}_mask_blur.jpg")

                if isfile(f"{mask_folder_path}/{image_id}_{opt_num}.jpg"):  # ?????                
                    image_file = glob.glob(f"{mask_folder_path}/{image_id}_{opt_num}.jpg")
                    for l in image_file:
                        source = l # source directory
                        destination = path
                        shutil.copy2(source, destination)
                else:
                    image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
                    for l in image_file:
                        source = l # source directory
                        destination = path
                        shutil.copy2(source, destination)
        except:
            pass

In [33]:
image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"
mask_image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur"

In [34]:
image_folder_list = glob.glob(f"{image_path}/*")
image_folder_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K_2',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur']

#### Visual Genome Word Lemma Translate Data

In [35]:
#Pos_Tag = "VERB" # NOUN, VERB, ADJ, ADV, NUM, PRON, CCONJ, ADP, AUX

In [37]:
df_genome_word_lemma_concat = pd.read_csv(f"{word_lemma_data_path}/Visual_Genome_{file_ext}_Word_Lemma_Search_Result.csv")
df_genome_word_lemma_concat

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994,2920126305
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,129190150113
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,123317182233
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861,4615142168
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866,54188097
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


##### Noun

In [38]:
df_genome_word_lemma_concat_noun = df_genome_word_lemma_concat[df_genome_word_lemma_concat["POS1"] == "NOUN"]
df_genome_word_lemma_concat_noun

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
2320,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2321637,1308718190
2321,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2412863,2404701598
2322,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what this is,2388466,34576516
2323,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what is not allowed,2399091,553912620
2324,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,"what , white",2408062,365513498
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [39]:
df_genome_word_lemma_concat_noun_select = df_genome_word_lemma_concat_noun.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_noun_select

,word,image_id
0,al,True
1,alabilir,True
2,alacağım,True
3,aldı,True
4,aldım,True
...,...,...
274,çık,True
275,çıktı,True
276,öldü,True
277,öldürmek,True


In [40]:
noun_word_select_list = df_genome_word_lemma_concat_noun_select[df_genome_word_lemma_concat_noun_select["image_id"] == True]["word"].to_list()
#noun_word_select_list

In [41]:
df_genome_word_lemma_concat_noun_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(noun_word_select_list)]
df_genome_word_lemma_concat_noun_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
2320,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2321637,1308718190
2321,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2412863,2404701598
2322,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what this is,2388466,34576516
2323,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what is not allowed,2399091,553912620
2324,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,"what , white",2408062,365513498
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [42]:
noun_word_list = list(set(df_genome_word_lemma_concat_noun_select_result["word"]))
noun_lemma_list = list(set(df_genome_word_lemma_concat_noun_select_result["lemma.spacy"]))

In [43]:
df_genome_noun_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_noun_select_result, noun_word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_noun_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352672,0
1,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352048,0
2,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352003,0
3,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2351969,0
4,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2351823,0
...,...,...,...,...,...,...,...,...,...,...,...
50781,VERB,NaN,söyle,söyle,söyle,say,say,443341,to say no cars are allowed,2328342,0
50782,VERB,NaN,söyle,söyle,söyle,say,say,443341,banana with just say no on it,2389787,0
50783,VERB,NaN,söyle,söyle,söyle,say,say,443341,large words on the road that say do not block way,2390001,0
50784,VERB,NaN,söyle,söyle,söyle,say,say,443341,one would say there are multiple patterns on t...,2358268,0


In [44]:
df_genome_noun_sample_result["lemma.spacy"].nunique()

279

In [ ]:
noun_output_path_folder = f"/home/kurubal/Downloads/Image_Select/NOUN"

Path(noun_output_path_folder).mkdir(parents=True, exist_ok=True)

In [47]:
# create folder
for lemma in noun_lemma_list:
    path = f"{noun_output_path_folder}/ALL_NOUN_FOLDER/{lemma}"
    Path(path).mkdir(parents=True, exist_ok=True)
    df_var = df_genome_noun_sample_result[df_genome_noun_sample_result["lemma.spacy"] == f"{lemma}"]
    for word in set(df_var["word"]):
        path2 = f"{path}/{word}"
        Path(path2).mkdir(parents=True, exist_ok=True)

In [48]:
# for lemma
create_word_folder_and_copy_image(df_genome_noun_sample_result, noun_lemma_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path, noun_output_path_folder)

##### Verb

In [49]:
df_genome_word_lemma_concat_verb = df_genome_word_lemma_concat[df_genome_word_lemma_concat["POS1"] == "VERB"]
df_genome_word_lemma_concat_verb

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
2320,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2321637,1308718190
2321,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2412863,2404701598
2322,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what this is,2388466,34576516
2323,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what is not allowed,2399091,553912620
2324,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,"what , white",2408062,365513498
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [50]:
df_genome_word_lemma_concat_verb_select = df_genome_word_lemma_concat_verb.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_verb_select

,word,image_id
0,al,True
1,alabilir,True
2,alacağım,True
3,aldı,True
4,aldım,True
...,...,...
274,çık,True
275,çıktı,True
276,öldü,True
277,öldürmek,True


In [51]:
verb_word_select_list = df_genome_word_lemma_concat_verb_select[df_genome_word_lemma_concat_verb_select["image_id"] == True]["word"].to_list()
#verb_word_select_list

In [52]:
df_genome_word_lemma_concat_verb_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(verb_word_select_list)]
df_genome_word_lemma_concat_verb_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
2320,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2321637,1308718190
2321,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2412863,2404701598
2322,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what this is,2388466,34576516
2323,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what is not allowed,2399091,553912620
2324,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,"what , white",2408062,365513498
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [53]:
verb_word_list = list(set(df_genome_word_lemma_concat_verb_select_result["word"]))
verb_lemma_list = list(set(df_genome_word_lemma_concat_verb_select_result["lemma.spacy"]))

In [54]:
df_genome_verb_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_verb_select_result, verb_word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_verb_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352672,0
1,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352048,0
2,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352003,0
3,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2351969,0
4,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2351823,0
...,...,...,...,...,...,...,...,...,...,...,...
50781,VERB,NaN,söyle,söyle,söyle,say,say,443341,to say no cars are allowed,2328342,0
50782,VERB,NaN,söyle,söyle,söyle,say,say,443341,banana with just say no on it,2389787,0
50783,VERB,NaN,söyle,söyle,söyle,say,say,443341,large words on the road that say do not block way,2390001,0
50784,VERB,NaN,söyle,söyle,söyle,say,say,443341,one would say there are multiple patterns on t...,2358268,0


In [56]:
df_genome_verb_sample_result["lemma.spacy"].nunique()

73

In [ ]:
verb_output_path_folder = f"/home/kurubal/Downloads/Image_Select/VERB"

Path(verb_output_path_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
# create folder
for lemma in verb_lemma_list:
    path = f"{verb_output_path_folder}/ALL_VERB_FOLDER/{lemma}"
    Path(path).mkdir(parents=True, exist_ok=True)
    df_var = df_genome_noun_sample_result[df_genome_noun_sample_result["lemma.spacy"] == f"{lemma}"]
    for word in set(df_var["word"]):
        path2 = f"{path}/{word}"
        Path(path2).mkdir(parents=True, exist_ok=True)

In [ ]:
# for lemma
create_word_folder_and_copy_image(df_genome_verb_sample_result, verb_lemma_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path, verb_output_path_folder)

##### Adjective Adverb Pronouns

In [57]:
df_genome_word_lemma_concat_adj_adv_pron = df_genome_word_lemma_concat[(df_genome_word_lemma_concat["POS1"] == "ADJ") | (df_genome_word_lemma_concat["POS1"] == "ADV") | (df_genome_word_lemma_concat["POS1"] == "PRON")]
df_genome_word_lemma_concat_adj_adv_pron

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num


In [ ]:
df_genome_word_lemma_concat_adj_adv_pron_select = df_genome_word_lemma_concat_adj_adv_pron.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_adj_adv_pron_select

,word,image_id
0,al,True
1,alabilir,True
2,alacağım,True
3,aldı,True
4,aldım,True
...,...,...
274,çık,True
275,çıktı,True
276,öldü,True
277,öldürmek,True


In [ ]:
adj_adv_pron_word_select_list = df_genome_word_lemma_concat_adj_adv_pron_select[df_genome_word_lemma_concat_adj_adv_pron_select["image_id"] == True]["word"].to_list()
#adj_adv_pron_word_select_list

In [ ]:
df_genome_word_lemma_concat_adj_adv_pron_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(adj_adv_pron_word_select_list)]
df_genome_word_lemma_concat_adj_adv_pron_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
2320,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2321637,1308718190
2321,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2412863,2404701598
2322,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what this is,2388466,34576516
2323,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what is not allowed,2399091,553912620
2324,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,"what , white",2408062,365513498
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [ ]:
adj_adv_pron_word_list = list(set(df_genome_word_lemma_concat_adj_adv_pron_select_result["word"]))
adj_adv_pron_lemma_list = list(set(df_genome_word_lemma_concat_adj_adv_pron_select_result["lemma.spacy"]))

In [ ]:
df_genome_adj_adv_pron_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_adj_adv_pron_select_result, adj_adv_pron_word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_adj_adv_pron_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352672,0
1,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352048,0
2,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2352003,0
3,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2351969,0
4,VERB,NEG,yoktur,yok,yok,there is no,none,90592,none,2351823,0
...,...,...,...,...,...,...,...,...,...,...,...
50781,VERB,NaN,söyle,söyle,söyle,say,say,443341,to say no cars are allowed,2328342,0
50782,VERB,NaN,söyle,söyle,söyle,say,say,443341,banana with just say no on it,2389787,0
50783,VERB,NaN,söyle,söyle,söyle,say,say,443341,large words on the road that say do not block way,2390001,0
50784,VERB,NaN,söyle,söyle,söyle,say,say,443341,one would say there are multiple patterns on t...,2358268,0


In [ ]:
df_genome_adj_adv_pron_sample_result["word"].nunique()

279

In [ ]:
adv_adj_pron_output_path_folder = f"/home/kurubal/Downloads/Image_Select/ADJ_ADV_PRON"

Path(adv_adj_pron_output_path_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
# create folder
for lemma in adj_adv_pron_lemma_list:
    path = f"{adv_adj_pron_output_path_folder}/ALL_ADJ_ADV_PRON_FOLDER/{lemma}"
    Path(path).mkdir(parents=True, exist_ok=True)
    df_var = df_genome_noun_sample_result[df_genome_noun_sample_result["lemma.spacy"] == f"{lemma}"]
    for word in set(df_var["word"]):
        path2 = f"{path}/{word}"
        Path(path2).mkdir(parents=True, exist_ok=True)

In [ ]:
# for lemma
create_word_folder_and_copy_image(df_genome_adj_adv_pron_sample_result, adj_adv_pron_lemma_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path, adv_adj_pron_output_path_folder)

In [ ]:
# create folder
# for word in word_list:
#    path = f"{output_path_folder}/ALL_{Pos_Tag}_FOLDER/{word}"
#    Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
# for word
#create_word_folder_and_copy_image(df_genome_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path ,output_path_folder)

In [ ]:
## Check Result 1
#df_genome_word_lemma_concat_coor = pd.read_csv(f"{word_lemma_data_path2}/Visual_Genome_{file_ext}_Word_Lemma_Coordinate_Search_Result.csv")
#df_genome_word_lemma_concat_coor = df_genome_word_lemma_concat_coor.drop(["height","width","x_koor","y_koor"], axis=1)
#df_genome_word_lemma_concat_coor

In [ ]:
# Check Result 2
#df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat_select_result,df_genome_word_lemma_concat_coor,how="left", on=["word","image_id","word_en_translate","lemma_en_translate","lemma.spacy","stem","num"])
##df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat, df_genome_word_lemma_concat_coor,how="left", on=["word","image_id"])
#df_genome_word_lemma_concat_select_result_coor_merge = df_genome_word_lemma_concat_select_result_coor_merge.drop_duplicates()
##df_genome_word_lemma_concat_select_result_coor_merge["num"] = df_genome_word_lemma_concat_select_result_coor_merge["num"].fillna(0)
#df_genome_word_lemma_concat_select_result_coor_merge

In [ ]:
#df_genome_word_lemma_concat_select_result_coor_merge["word"].nunique()

In [ ]:
#word_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["word"]))
#lemma_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["lemma.spacy"]))

In [ ]:
#df_genome_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_select_result_coor_merge, word_list, "word", 
#                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
#df_genome_sample_result

In [ ]:
#df_genome_sample_result["word"].nunique()